In [1]:
import json
from operator import index
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import pickle
import time

In [2]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import ExtraTreesRegressor
import xgboost as xgb
#import tensorflow as tf

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, RobustScaler

from sklearn.model_selection import train_test_split, cross_validate, ShuffleSplit

from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.metrics import median_absolute_error

import warnings
warnings.filterwarnings('once', '.*sliced data.*', )
warnings.filterwarnings('once', '.*Maximum iterations*', )

/home/max/.local/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [3]:
def prepare_dataset(path : str, workload_name : str):
    dataset = {}
    files = os.listdir(path)

    for file in files:
        with open(path+"/"+file, "r") as f:
            raw_str = f.read()
        data = json.loads(raw_str)

        if "layers" in data.keys():
            del data["layers"]

        if "relay" in data.keys():
            del data["relay"]

        if "conv2d" in workload_name:
            data["kernel_0"] = data["kernel"][0]
            data["kernel_1"] = data["kernel"][1]
            del data["kernel"]
            data["dilation_0"] = data["dilation"][0]
            data["dilation_1"] = data["dilation"][1]
            del data["dilation"]

            if not "kernel layout" in data.keys():
                data["kernel layout"] = "OIHW"

        elif workload_name == "dense":
            data["features"] = data["input shape"][1]
            del data["input shape"]
            del data["output shape"]
            #print()

        elif workload_name in ["max_pool2d", "avg_pool2d"]:
            data["pool_0"] = data["pool_size"][0]
            data["pool_1"] = data["pool_size"][1]
            del data["pool_size"]

        if workload_name in ["max_pool2d", "avg_pool2d", "conv2d", "dilated_conv2d", "depthwise_conv2d"]:
            del data["padding"]
            data["C_I"] = data["input shape"][3]
            data["H_I"] = data["input shape"][1]
            data["W_I"] = data["input shape"][2]
            del data["input shape"]

            data["C_O"] = data["output shape"][3]
            data["H_O"] = data["output shape"][1]
            data["W_O"] = data["output shape"][2]
            del data["output shape"]

            key = "strides"
            if "stride" in data.keys():
                key = "stride"
            #print(key)
            data["strides_0"] = data[key][0]
            data["strides_1"] = data[key][1]
            del data["strides"]
            if "stride" in data.keys():
                del data["stride"]

        dataset[file] = data

    return dataset

def create_dataframe(dataset : dict, workload_name : str, labels = ["time", "power", "memory"]):
    df = pd.DataFrame.from_dict(dataset, orient='index')
    categoricals = [
        "output dtype",
        "compute dtype",
        "workload",
        ]
    if workload_name in ["conv2d", "max_pool2d", "avg_pool2d", "depthwise_conv2d", "dilated_conv2d"]:
        if "data_layout" in list(df.columns):
            categoricals += [
                #"padding",
                "data_layout",
            ]
        else:    
            categoricals += [
                #"padding",
                "data layout",
            ]
        

    if "conv2d" in workload_name:
        if "kernel_layout" in list(df.columns):
            categoricals += [
                "kernel_layout",
            ]
        if "kernel layout" in list(df.columns):
            categoricals += [
                "kernel layout",
            ]

    for col in categoricals:
        oh = pd.get_dummies(df[col], prefix=col, drop_first=False)
        df = pd.concat([df, oh], axis=1).drop(col, axis=1)
    
    features = list(df.columns)
    #labels = ["time", "power", "memory"]
    for label in labels:
        idx = features.index(label)
        del features[idx]
    del idx
    df = df.drop_duplicates(subset=features)

    label_data = []
    for label in labels:
        label_data.append(df[label])
        df.pop(label)

    output = pd.concat(label_data, axis=1)

    return df, output

In [4]:
!ls dataset/

 cuda_980ti			     'llvm -mcpu=haswell_i7-4790'
 cuda_A100			      opencl_Mali-T628_MP6
 cuda_K80			      opencl_Mali-T628_MP6_no_power
'llvm -mcpu=haswell_E5-2680'	      rasp4b
'llvm -mcpu=haswell_E5-2680_faulty'


In [5]:
dataset_base = "./dataset"
targets = os.listdir(dataset_base)

layer_targets = list()
for target in targets:
    target_path = dataset_base + "/" + target
    layers = os.listdir(target_path)
    
    for layer in layers:
        dataset_path = target_path + "/" + layer + "/"
        print(dataset_path)
        layer_targets.append(dataset_path)

print("found {0} folders with samples, going to train models for each of these targets".format(len(layer_targets)))
print()

./dataset/cuda_980ti/avg_pool2d/
./dataset/cuda_980ti/conv2d/
./dataset/cuda_980ti/dense/
./dataset/cuda_980ti/depthwise_conv2d/
./dataset/cuda_980ti/dilated_conv2d/
./dataset/cuda_980ti/max_pool2d/
./dataset/cuda_A100/avg_pool2d/
./dataset/cuda_A100/conv2d/
./dataset/cuda_A100/dense/
./dataset/cuda_A100/depthwise_conv2d/
./dataset/cuda_A100/dilated_conv2d/
./dataset/cuda_A100/max_pool2d/
./dataset/cuda_K80/avg_pool2d/
./dataset/cuda_K80/conv2d/
./dataset/cuda_K80/dense/
./dataset/cuda_K80/depthwise_conv2d/
./dataset/cuda_K80/dilated_conv2d/
./dataset/cuda_K80/max_pool2d/
./dataset/llvm -mcpu=haswell_E5-2680/avg_pool2d/
./dataset/llvm -mcpu=haswell_E5-2680/conv2d/
./dataset/llvm -mcpu=haswell_E5-2680/dense/
./dataset/llvm -mcpu=haswell_E5-2680/depthwise_conv2d/
./dataset/llvm -mcpu=haswell_E5-2680/dilated_conv2d/
./dataset/llvm -mcpu=haswell_E5-2680/max_pool2d/
./dataset/llvm -mcpu=haswell_E5-2680_faulty/conv2d/
./dataset/llvm -mcpu=haswell_E5-2680_faulty/dense/
./dataset/llvm -mcpu=ha

In [6]:
layer_targets = [
"./dataset/cuda_980ti/avg_pool2d/",
"./dataset/cuda_980ti/conv2d/",
"./dataset/cuda_980ti/dense/",
"./dataset/cuda_980ti/depthwise_conv2d/",
"./dataset/cuda_980ti/dilated_conv2d/",
"./dataset/cuda_980ti/max_pool2d/",
]

In [7]:
models = {
    "xgb": xgb.XGBRegressor,
    "ert": ExtraTreesRegressor,
    "dTr": DecisionTreeRegressor,
    "liR": LinearRegression,
    "kNN": KNeighborsRegressor,
    "SVR": SVR,
    "MLP": MLPRegressor,
}

kwargs_dict = {
    "xgb": None,
    "ert": {"n_estimators":175},
    "dTr": {"criterion":"mae", },
    "liR": {"normalize": True, "n_jobs":-1},
    "kNN": {"n_neighbors":3, "weights":"distance"},
    "SVR": {},
    "MLP": {"hidden_layer_sizes":(100,400,200,100,), "max_iter":500},
}

results = {}

In [ ]:
for target in layer_targets:
    print(target)
    tmp = target.split("/")
    workload_name = tmp[-2]
    device_name = tmp[-3]
    files = os.listdir(target)
    dataset = {}
    
    labels = ["time", "power", "memory"]
    if device_name == "rasp4b" or device_name == "opencl_Mali-T628_MP6":
        labels = ["time", "power", "ws_size", "io_size"]
    
    print("{} : {}\t:\t contains {} samples".format(workload_name, device_name, len(files)))
    dataset = prepare_dataset(target, workload_name)
    df, output = create_dataframe(dataset, workload_name, labels=labels)
    print("\t[INFO] Remaining Samples after duplicate elimination:\t{}".format(len(df)))
    X = df.to_numpy()
    Y = output.to_numpy()
    labels = list(output.columns)

    X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.2)
    steps = 25
    start = step_size = len(X_train)//steps
    end = len(X_train)
    #scoring = ["r2", "neg_mean_absolute_error", "neg_mean_absolute_percentage_error", "max_error", "neg_median_absolute_error"]
    for method, constructor in models.items():
        for idx, name in enumerate(labels):
            for train_size in range(start, end, step_size):
                print(device_name)
                print(workload_name)
                print(name)
                print(method)
                print(train_size)

                if kwargs_dict[method] != None:
                    model = make_pipeline(RobustScaler(), constructor(**kwargs_dict[method]))
                else:
                    model = make_pipeline(RobustScaler(), constructor())

                model.fit(X_train[0:train_size], y_train[0:train_size,idx])
                pred_test = model.predict(X_test)
                r2_test = r2_score(y_test[:,idx], pred_test)
                max_error = np.max(np.abs(pred_test))
                mae = mean_absolute_error(y_test[:,idx], pred_test)
                mape = mean_absolute_percentage_error(y_test[:,idx], pred_test)
                medae = median_absolute_error(y_test[:,idx], pred_test)
                result = {
                    "device" : device_name,
                    "workload" : workload_name,
                    "metric" : name,
                    "predictor" : method,
                    "r2" : r2_test,
                    "max_error" : max_error,
                    "mae" : mae,
                    "mape" : mape,
                    "medae" : medae,
                    "dataset_min" : y_train[0:train_size,idx].min(),
                    "dataset_max" : y_train[0:train_size,idx].max(),
                    "dataset_mean" : y_train[0:train_size,idx].mean(),
                    "dataset_median" : np.median(y_train[:,idx]),
                    "dataset_size": train_size,
                }
                print(result)
                #print("R2: {.3f}\tMAE {.3f}\tMAPE {.3f}\tMEDAE {.3f}\tMaxError {.3f}".format(result["r2"],result["mae"],result["mape"],result["medae"],result["max_error"]))
                results[device_name+"-"+workload_name+"-"+name+"-"+method+":"+str(train_size)] = result

./dataset/cuda_980ti/avg_pool2d/
avg_pool2d : cuda_980ti	:	 contains 4692 samples
	[INFO] Remaining Samples after duplicate elimination:	4692
cuda_980ti
avg_pool2d
time
xgb
150
{'device': 'cuda_980ti', 'workload': 'avg_pool2d', 'metric': 'time', 'predictor': 'xgb', 'r2': 0.6257710168089552, 'max_error': 15.85395, 'mae': 1.577896561294979, 'mape': 1.9005720069056025, 'medae': 0.7851575223541256, 'dataset_min': 0.00764443, 'dataset_max': 22.7072, 'dataset_mean': 3.7938289555333338, 'dataset_median': 1.45204, 'dataset_size': 150}
cuda_980ti
avg_pool2d
time
xgb
300
{'device': 'cuda_980ti', 'workload': 'avg_pool2d', 'metric': 'time', 'predictor': 'xgb', 'r2': 0.7546991988212559, 'max_error': 17.626747, 'mae': 1.2195044549344791, 'mape': 1.408373192533336, 'medae': 0.5732872058105469, 'dataset_min': 0.006952310000000001, 'dataset_max': 27.1845, 'dataset_mean': 3.511813319866667, 'dataset_median': 1.45204, 'dataset_size': 300}
cuda_980ti
avg_pool2d
time
xgb
450
{'device': 'cuda_980ti', 'workl

In [ ]:
date_time = "size_investigation/results_rbst_scaler_980ti_"+str(int(time.time()))
results = pd.DataFrame.from_dict(results, "index")

In [ ]:
results.to_csv(date_time+"-predictor_results.csv")
results.to_excel(date_time+"-predictor_results.xlsx")
results.to_html(date_time+"-predictor_results.html")
results.to_markdown(date_time+"-predictor_results.md")
print("done")

In [ ]:
results

In [ ]:
viz = results.set_index('dataset_size', inplace=False)
viz[(viz.metric == "time")  & (viz.workload == "conv2d")].groupby('predictor')["r2"].plot(kind="line", legend=True, figsize=(15,5))

#groupby('predictor').plot(legend=True)